In [3]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from IPython.display import display, HTML

In [4]:
website = requests.get("https://www.timesjobs.com/candidate/job-search.html?searchType=personalizedSearch&from=submit&searchTextSrc=ft&searchTextText=%22Data+Analysis%22&txtKeywords=%22Data+Analysis%22%2C&cboWorkExp1=1&txtLocation=").text

soup = bs(website, 'html.parser')

In [15]:
df = pd.DataFrame(columns=['Job_Title','Day_Posted','Company_Name','Skill_Required','Job_Description','Job_Link'])
job_titles = soup.find_all('h2', class_='heading-trun')
day_posted = soup.find_all('span', class_='sim-posted')
company_names = soup.find_all('h3', class_='joblist-comp-name')
skills = soup.find_all('div', class_='more-skills-sections')
job_descriptions = soup.find_all('li', class_='job-description__')
job_link = soup.find_all('li', class_='clearfix job-bx wht-shd-bx')

for job_title, day_of_post, company, skill, description, link in zip(job_titles, day_posted, company_names, skills, job_descriptions, job_link):
    a_tag = link.find('a')
    job_title = job_title.get_text(strip=True)
    posted = day_of_post.get_text(strip=True)
    company_name = company.get_text(strip=True)
    skill_require = skill.get_text(strip=True)
    job_description = description.get_text(strip=True)
    job_link = a_tag['href'] if a_tag else 'No Link'
    df.loc[len(df)] = [job_title, posted, company_name, skill_require, job_description, job_link]

In [14]:
def format_for_display(x):
    """Transform link into HTML anchor tag for display."""
    if x != 'No Link':
        return '<a href="' + x + '" target="_blank">' + x + '</a>'
    else:
        return x

df['Job_Link_Display'] = df['Job_Link'].apply(format_for_display)
df['Job_Link_CSV'] = df['Job_Link']
display(HTML(df.to_html(columns=['Job_Title', 'Day_Posted', 'Company_Name', 'Skill_Required', 'Job_Description', 'Job_Link_Display'], escape=False)))
df.to_csv(r'C:\Users\arzag\Desktop\Web Scraping\data_analysis_jobs.csv', columns=['Job_Title', 'Day_Posted', 'Company_Name', 'Skill_Required', 'Job_Description', 'Job_Link_CSV'], index=False)


,Job_Title,Day_Posted,Company_Name,Skill_Required,Job_Description,Job_Link_Display
0,"Cluster Manager , Banca",1 day ago,JOBS N TA HR SERVICES,Life insuranceteam HandlingInsurancebancaassurance,Key Responsibilities:Sales Leadership and Target Achievement:Drive life insurance sales across all branches within the cluster to achieve business tar...,https://www.timesjobs.com/job-detail/cluster-manager-banca-jobs-n-ta-hr-services-chennai-coimbatore-tirunelveli-1-to-5-yrs-jobid-OlvwhombFk5zpSvf__PLUS__uAgZw==&source=srp
1,Data Analyst,1 day ago,AppSoft Solutions,Data Analysisdata analystStatistical ModelingTime Series AnalysisData IntegrationData ArchitectureEnterprise Data Modeling,Data Analyst / Senior Data Analyst / Data Analyst - Tech LeadVery Urgent Requirement If Your Profile Matches The Above Requirement And If You're I...,https://www.timesjobs.com/job-detail/data-analyst-appsoft-solutions-bengaluru-bangalore-chennai-delhi-mumbai-pune-0-to-3-yrs-jobid-fZyJE9YXWd1zpSvf__PLUS__uAgZw==&source=srp
2,Data Analyst Internship in Pune,few days ago,Maxgen Technologies,data analysis,Maxgen Technologies Pvt ltd is based in pune offering aninternship. we are providing in data analysis internship and deep learningprogram..why our ser...,https://www.timesjobs.com/job-detail/data-analyst-internship-in-pune-maxgen-technologies-pune-amravati-dhule-sangli-satara-0-to-1-yrs-jobid-__SLASH__9yUiiWOWahzpSvf__PLUS__uAgZw==&source=srp
3,Data Analyst Internship in Ahmedabad,few days ago,Maxgen Technologies,internshipdata analysis,"Maxgen Technologies Pvt ltd is an it company since 2014 in ahmedabad , . We are offering data analyst and power biprogram.what we are serving?professi...",https://www.timesjobs.com/job-detail/data-analyst-internship-in-ahmedabad-maxgen-technologies-ahmedabad-anand-gandhinagar-mehsana-surat-0-to-1-yrs-jobid-f7yhHZvBsvFzpSvf__PLUS__uAgZw==&source=srp
4,Officer Financial &DataAnalysisRegulations Research Product Associate - C10 - Mumbai at Citi,few days ago,Citibank India,financial analysisdata monitoringproblem solvingprocess improvementcommunication skillsstrong communication skillsliaisonmanagementaccommodation,"Officer Financial & amp; Data Analysis Regulations Research Product Associate - C10 - MumbaiJob Req ID: 25821282Location ( s ) : Mumbai , IndiaJo...",https://www.timesjobs.com/job-detail/officer-financial-data-analysis-regulations-research-product-associate-c10-mumbai-at-citi-citibank-india-mumbai-1-to-5-yrs-jobid-T52Fbz4L9nJzpSvf__PLUS__uAgZw==&source=srp
5,Sales Coordinator,few days ago,decent manpower,CommunicationOrganizational skillsCRM software proficiencyData analysisCustomer service excellence,JOB DESCRIPTION Proven experience as a sales coordinator. Responsible to prepare MIS reports. coordinate with the sales & service department team....,https://www.timesjobs.com/job-detail/sales-coordinator-decent-manpower-ahmedabad-1-to-5-yrs-jobid-Cv1alhfwb8hzpSvf__PLUS__uAgZw==&source=srp
6,Data Administrator,a month ago,Rahul jatav,Business AnalysisData AnalysisData Base AdministrationCall HR 7669453732Human ResourcsIT RecruitmentStaffing,"We are searching for an IT analyst to assist with the development , maintenance , and implementation of IT solutions for our company. In this role ,...",https://www.timesjobs.com/job-detail/data-administrator-airlines-airport-calicut-kozhikode-cochin-kochi-ernakulam-palakkad-thiruvananthapuram-thrissur-0-to-3-yrs-jobid-__SLASH__TG0ryTDOspzpSvf__PLUS__uAgZw==&source=srp
7,Urgent Hiring For IT Analyst,a month ago,Rahul jatav,Business AnalysisData AnalysisData Base AdministrationCall HR 7669453732Human ResourcsIT RecruitmentStaffing,"We are searching for an IT analyst to assist with the development , maintenance , and implementation of IT solutions for our company. In this role ,...",https://www.timesjobs.com/job-detail/urgent-hiring-for-it-analyst-airlines-airport-calicut-kozhikode-cochin-kochi-ernakulam-kottayam-thiruvananthapuram-thrissur-0-to-3-yrs-jobid-UkR4qoDkoylzpSvf__PLUS__uA